In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


data = pd.read_excel("preprocessed-dataset.xlsx")
target_attribute = data['preprocessed']

X_train, X_test, y_train, y_test = train_test_split(data, target_attribute, test_size=0.10)

In [2]:
print("Seluruh Data",(len(data)))
print("X_Train",(len(X_test)))
print("X_Test",(len(X_train)))
print("Y_Test",(len(y_test)))
print("Y_Train",(len(y_train)))

Seluruh Data 14321
X_Train 1433
X_Test 12888
Y_Test 1433
Y_Train 12888


In [4]:
dokumen = X_test['preprocessed'].astype(str)
dokumen = dokumen.dropna(axis=0, how='any')
query = X_train['preprocessed'].astype(str)
query = query.dropna(axis=0, how='any')

In [5]:
from sklearn.feature_extraction.text import CountVectorizer # tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer # tf-idf
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords # preprocessing\
from math import*
from numpy import linalg as LA
import numpy as np
class Engine:
    def __init__(self):
        self.cosine_score = []
        self.train_set = []  # Documents
        self.test_set = []  # Query

    def addDocument(self, word): # fungsi untuk menambahkan dokumen dataset ke dalam list train_set
        self.train_set.append(word)

    def setQuery(self, word):  # fungsi untuk menambahkan data query ke dalam list test_Set
        self.test_set.append(word)

    def process_score(self):
        vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, max_df = 0.5, smooth_idf=True)
        svd_model = TruncatedSVD(n_components=100,algorithm='randomized',n_iter=10)
        lsa = Pipeline([('tfidf', vectorizer),('svd', svd_model)])
        #transformer = TfidfTransformer(stop_words='english', use_idf=True, smooth_idf=True)

        trainVectorizerArray = lsa.fit_transform(self.train_set).tolist() 
        # menghitung Bobot dokumen dataset dan uji dan kemudian disimpan dalam bentuk array 
        testVectorizerArray = lsa.transform(self.test_set).tolist()

        cx = lambda a, b: round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 3) 
        #fungsi tanpa nama untuk normalisasi data dan definisi rumus Cosine Similarity 
        #         print testVectorizerArray
        output = []
        for i in range(0, len(testVectorizerArray)):
            output.append([])

        for vector in trainVectorizerArray:
            # print vector
            u = 0
            for testV in testVectorizerArray:
                #perhitungan Cosine Similarity dalam bentuk vector dari dataset dengan query
                #yang di masukan yang kemudian mengembalikan nilai cosine ke dalam variable
                #cosine_score dalam bentuk list.
                # print testV
                cosine = cx(vector, testV)
                #                 self.cosine_score.append(cosine)
                #                 bulatin = (round(cosine),2)
                output[u].append((cosine))
                u = u + 1
        return output
        # return testVectorizerArray

In [26]:
engine = Engine()

docs = [str(x) for x in dokumen]
documentNames = list()

for i, doc in enumerate (docs):
    engine.addDocument(doc) 
    documentNames.append("Document_{}".format(i+1))

for queries in query:
    engine.setQuery(queries) #inputandata uji

titles_score = engine.process_score()
ScoreDf = (pd.DataFrame(titles_score)).T
ScoreDf.columns = X_train['preprocessed']
ScoreDf["Documents"] = documentNames
ScoreDf["Reviewer"] = X_test["Reviewer"].values
ScoreDf["Judul"] = X_test["Judul"].values
ScoreDf

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


preprocessed,method prioritize content,prune decision tree misclassification cost,segmentation dimensionality reduction,multi label classification method multi target regression,freshness matter flower food web authority,rule discovery time series,annosearch image auto annotation search,answer aggregate keyword query relational database minimal group bys,indri trec 2004 terabyte track,graph regularize feature selection datum reconstruction,...,method surprise temporal pattern,pagerank hit unify framework link analysis,datum stream algorithm compute property large graph,overview inex 2011 book social search track,road knowledge,locate relevant text xml document,leverage count sample hide database,Documents,Reviewer,Judul
0,-0.019,-0.008,0.094,-0.023,0.006,0.005,0.017,-0.029,-0.008,0.008,...,0.206,-0.016,-0.010,-0.004,0.192,-0.054,-0.012,Document_1,fosca giannotti,discovering the geographical borders of human ...
1,0.024,-0.006,-0.065,-0.004,0.336,-0.010,0.182,0.009,0.011,0.000,...,0.029,0.069,-0.009,0.134,0.009,0.219,-0.016,Document_2,marijn koolen,"using anchor text, spam filtering and wikipedi..."
2,-0.000,-0.009,-0.027,-0.000,-0.004,-0.017,-0.002,0.003,0.002,-0.005,...,-0.018,0.006,0.230,-0.003,0.013,-0.003,0.020,Document_3,mohammed zaki,genmax: an efficient algorithm for mining maxi...
3,-0.011,-0.014,-0.010,0.023,0.000,0.008,-0.004,0.027,-0.013,0.020,...,0.036,-0.003,0.020,-0.021,0.003,-0.024,0.001,Document_4,jeffrey xu yu,efficient computation of the skyline cube
4,-0.021,-0.007,-0.035,-0.007,0.001,0.006,0.010,-0.027,-0.012,0.013,...,-0.012,0.004,0.005,0.327,0.014,0.010,0.016,Document_5,panayiotis tsaparas,opinion maximization in social networks
5,0.365,0.013,0.044,0.218,0.008,-0.004,-0.012,0.025,-0.007,0.007,...,0.339,0.013,0.003,-0.006,0.019,-0.028,-0.021,Document_6,fabio pinelli,individual mobility profiles: methods and appl...
6,0.006,-0.003,-0.015,0.003,0.011,-0.013,-0.006,-0.020,-0.008,0.156,...,0.016,-0.004,0.338,-0.002,0.002,-0.006,0.055,Document_7,jiawei han,classifying large data sets using svms with hi...
7,0.042,0.007,0.017,-0.061,0.215,-0.017,-0.279,0.128,0.015,-0.077,...,-0.160,0.023,-0.106,-0.208,-0.012,0.001,-0.014,Document_8,bo long,treatment of meat by capacitor discharge
8,0.317,0.003,0.094,0.182,-0.001,0.000,-0.005,-0.022,0.003,0.003,...,0.314,-0.001,0.005,-0.010,-0.017,-0.001,0.002,Document_9,charu c. aggarwal,methods and apparatus for user-centered simila...
9,0.260,-0.015,-0.030,0.140,-0.003,-0.005,0.005,0.304,0.008,-0.008,...,0.240,0.003,-0.005,-0.009,0.006,0.005,0.005,Document_10,bo long,system and method for probabilistic relational...


In [38]:
df_listed = list()
for i in query:
    labels = list()
    for j in ScoreDf[i]:
        if j>0.000:
            labels.append(1)
        else:
            labels.append(0)
    datadf = pd.DataFrame(ScoreDf[i])
    datadf['Documents'] = ScoreDf['Documents']
    datadf['Labels'] = labels
    datadf['Reviewer'] = ScoreDf['Reviewer'].values
    df_listed.append(datadf.sort_values(by=[i], ascending=False))

TypeError: '>' not supported between instances of 'str' and 'float'

In [ ]:
df_listed[0].head(5)